In [ ]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Método Canny Edge (Derivadas y Bordes)</h3>
    <video id="v_canny_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_canny_pro" width="640" height="480" style="background: #000; border: 2px solid #27ae60; border-radius: 10px;"></canvas>
    <div style="margin-top: 15px;">
        <button id="btn_on_canny" onclick="iniciarCannyPro()" style="padding: 10px 20px; background: #27ae60; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">🚀 ACTIVAR CANNY</button>
        <button id="btn_off_canny" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">🛑 APAGAR</button>
    </div>
    <p id="log_canny_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script>
// SISTEMA GLOBAL DE GESTIÓN DE CÁMARA
if (typeof window.cameraManager === 'undefined') {
    window.cameraManager = {
        currentStream: null,
        currentFilter: null,
        activeLoop: null
    };
}

function detenerTodo() {
    if (window.cameraManager.activeLoop) {
        window.cameraManager.activeLoop = false;
    }
    
    if (window.cameraManager.currentStream) {
        window.cameraManager.currentStream.getTracks().forEach(t => t.stop());
        window.cameraManager.currentStream = null;
    }
    
    // LIMPIAR TODOS LOS CANVAS
    ['c_canny_pro', 'c_out', 'c_sobel_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    ['btn_off_canny', 'btn_off_sobel', 'b_stop'].forEach(id => {
        const btn = document.getElementById(id);
        if (btn) btn.style.display = 'none';
    });
    
    ['btn_on_canny', 'btn_on_sobel', 'b_start'].forEach(id => {
        const btn = document.getElementById(id);
        if (btn) btn.style.display = 'inline-block';
    });
    
    ['log_canny_pro', 'log_sobel_pro', 'debug_log'].forEach(id => {
        const log = document.getElementById(id);
        if (log) log.innerText = "Cámara liberada.";
    });
    
    window.cameraManager.currentFilter = null;
}

// Implementación REAL de Canny Edge Detection
function applyCannyEdgeDetection(imageData) {
    const width = imageData.width;
    const height = imageData.height;
    const data = imageData.data;
    
    // 1. Convertir a escala de grises
    const gray = new Uint8ClampedArray(width * height);
    for (let i = 0; i < data.length; i += 4) {
        const idx = i / 4;
        gray[idx] = 0.299 * data[i] + 0.587 * data[i + 1] + 0.114 * data[i + 2];
    }
    
    // 2. Suavizado Gaussiano 5x5
    const smoothed = new Uint8ClampedArray(width * height);
    const gaussianKernel = [
        2, 4, 5, 4, 2,
        4, 9, 12, 9, 4,
        5, 12, 15, 12, 5,
        4, 9, 12, 9, 4,
        2, 4, 5, 4, 2
    ];
    const kernelSum = 159;
    
    for (let y = 2; y < height - 2; y++) {
        for (let x = 2; x < width - 2; x++) {
            let sum = 0;
            for (let ky = -2; ky <= 2; ky++) {
                for (let kx = -2; kx <= 2; kx++) {
                    const idx = (y + ky) * width + (x + kx);
                    sum += gray[idx] * gaussianKernel[(ky + 2) * 5 + (kx + 2)];
                }
            }
            smoothed[y * width + x] = sum / kernelSum;
        }
    }
    
    // 3. Calcular gradientes con Sobel
    const gradX = new Float32Array(width * height);
    const gradY = new Float32Array(width * height);
    const magnitude = new Float32Array(width * height);
    const direction = new Float32Array(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            // Sobel X
            const gx = (
                -smoothed[(y-1)*width + (x-1)] + smoothed[(y-1)*width + (x+1)] +
                -2*smoothed[y*width + (x-1)] + 2*smoothed[y*width + (x+1)] +
                -smoothed[(y+1)*width + (x-1)] + smoothed[(y+1)*width + (x+1)]
            );
            
            // Sobel Y
            const gy = (
                -smoothed[(y-1)*width + (x-1)] - 2*smoothed[(y-1)*width + x] - smoothed[(y-1)*width + (x+1)] +
                smoothed[(y+1)*width + (x-1)] + 2*smoothed[(y+1)*width + x] + smoothed[(y+1)*width + (x+1)]
            );
            
            gradX[idx] = gx;
            gradY[idx] = gy;
            magnitude[idx] = Math.sqrt(gx * gx + gy * gy);
            direction[idx] = Math.atan2(gy, gx);
        }
    }
    
    // 4. Supresión no-máxima (esto hace que Canny sea diferente de Sobel)
    const suppressed = new Float32Array(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            const angle = direction[idx] * 180 / Math.PI;
            const mag = magnitude[idx];
            
            let n1 = 0, n2 = 0;
            
            // Determinar vecinos según la dirección del gradiente
            if ((angle >= -22.5 && angle < 22.5) || (angle >= 157.5 || angle < -157.5)) {
                // Horizontal
                n1 = magnitude[idx - 1];
                n2 = magnitude[idx + 1];
            } else if ((angle >= 22.5 && angle < 67.5) || (angle >= -157.5 && angle < -112.5)) {
                // Diagonal /
                n1 = magnitude[(y-1)*width + (x+1)];
                n2 = magnitude[(y+1)*width + (x-1)];
            } else if ((angle >= 67.5 && angle < 112.5) || (angle >= -112.5 && angle < -67.5)) {
                // Vertical
                n1 = magnitude[(y-1)*width + x];
                n2 = magnitude[(y+1)*width + x];
            } else {
                // Diagonal \
                n1 = magnitude[(y-1)*width + (x-1)];
                n2 = magnitude[(y+1)*width + (x+1)];
            }
            
            // Suprimir si no es máximo local
            if (mag >= n1 && mag >= n2) {
                suppressed[idx] = mag;
            } else {
                suppressed[idx] = 0;
            }
        }
    }
    
    // 5. Umbralización con histéresis (doble umbral + seguimiento de bordes)
    const lowThreshold = 30;
    const highThreshold = 90;
    const edges = new Uint8ClampedArray(width * height);
    
    // Marcar píxeles fuertes
    for (let i = 0; i < suppressed.length; i++) {
        if (suppressed[i] >= highThreshold) {
            edges[i] = 255; // Borde fuerte
        } else if (suppressed[i] >= lowThreshold) {
            edges[i] = 128; // Borde débil (candidato)
        } else {
            edges[i] = 0;
        }
    }
    
    // Seguimiento de bordes (conectar bordes débiles a fuertes)
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            if (edges[idx] === 128) { // Borde débil
                // Verificar si está conectado a un borde fuerte
                let connected = false;
                for (let dy = -1; dy <= 1; dy++) {
                    for (let dx = -1; dx <= 1; dx++) {
                        if (edges[(y+dy)*width + (x+dx)] === 255) {
                            connected = true;
                            break;
                        }
                    }
                    if (connected) break;
                }
                
                edges[idx] = connected ? 255 : 0;
            }
        }
    }
    
    // Convertir a ImageData
    const output = new ImageData(width, height);
    for (let i = 0; i < edges.length; i++) {
        output.data[i * 4] = edges[i];
        output.data[i * 4 + 1] = edges[i];
        output.data[i * 4 + 2] = edges[i];
        output.data[i * 4 + 3] = 255;
    }
    
    return output;
}

async function iniciarCannyPro() {
    const log = document.getElementById('log_canny_pro');
    
    detenerTodo();
    
    try {
        log.innerText = "📷 Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'canny';
        
        const v = document.getElementById('v_canny_pro');
        const canvas = document.getElementById('c_canny_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        await new Promise(resolve => setTimeout(resolve, 500));
        
        document.getElementById('btn_on_canny').style.display = 'none';
        document.getElementById('btn_off_canny').style.display = 'inline-block';
        log.innerText = "✅ Detección de bordes Canny activa";
        
        window.cameraManager.activeLoop = true;
        let frameCount = 0;
        
        function processFrame() {
            if (!window.cameraManager.activeLoop || window.cameraManager.currentFilter !== 'canny') {
                return;
            }
            
            try {
                ctx.drawImage(v, 0, 0, canvas.width, canvas.height);
                const imageData = ctx.getImageData(0, 0, canvas.width, canvas.height);
                
                // Aplicar CANNY REAL
                const edges = applyCannyEdgeDetection(imageData);
                
                ctx.putImageData(edges, 0, 0);
                
                frameCount++;
                if (frameCount % 30 === 0) {
                    log.innerText = "✅ Canny activo - Frames: " + frameCount;
                }
                
                requestAnimationFrame(processFrame);
                
            } catch (e) {
                console.error('Error:', e);
                log.innerText = "❌ Error: " + e.message;
            }
        }
        
        processFrame();
        
    } catch (e) {
        log.innerText = "❌ Error: " + e.message;
        console.error(e);
    }
}
</script>